## Problem 4: What is the closest shopping center from your home / work? (5 points)

In the last problem you should find out the closest shopping center from two different locations a) your home and b) work place. 

**Steps**:

 - Create a txt-file called `activity_locations.txt` (use same formatting as in Problem 1) with two columns:
    - `id`: unique id for each row
    - `addr`:  address of your work and home (or any two addresses in the Helsinki Region.)
    
Save the text file into this repository.
    
 - Read those addresses using pandas and geocode the addresses.
 - Find out the nearest shopping center to these points using Shapely `nearest_points`.
 - Print out the name of the shopping center that is nearest to a) home and b) work. For example: `Shopping center closest to home: REDI`.

In [3]:
# read in input files
import geopandas as gpd
import pandas as pd

in_fp = r'C:\Users\paul-\Desktop\AutoGIS\Lesson3\Exercise\data\activity_locations.txt'

# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
try:
    data = pd.read_csv(in_fp, sep=';')
except:
    raise NotImplementedError()

data.head()

,id,name,addr
0,100,home,"Väinö Auerin katu 13, 00550 Helsinki, Finland"
1,101,work,"Nilsiänkatu 15, 00510 Helsinki, Finland"


In [5]:
# Geocode activity locations
from geopandas.tools import geocode 

geo = geocode(data['addr'], provider='Nominatim', user_agent='autogis_PH', timeout=4)

geo.head()

,geometry,address
0,POINT (24.96100 60.20523),"13, Väinö Auerin katu, Kumpula, Helsinki, Hels..."
1,POINT (24.94914 60.19585),"Vallila Interior, 15, Nilsiänkatu, Vallila, He..."


In [6]:
# Check coordinate reference system information (re-project/define if needed)
geo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
fp= r'C:\Users\paul-\Desktop\AutoGIS\Lesson3\Exercise\data\shopping_centers.shp'

shp = gpd.read_file(fp)

shp.head()

,id,name,address,addr,geometry
0,1000,Itis,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...","Itäkatu 7, 00930 Helsinki, Finland",POINT (25504598.602 6677662.109)
1,1001,Center,"Helsinki, 1, Kaivokatu, Keskusta, Kluuvi, Etel...","Kaivokatu 1, 00100 Helsinki, Finland",POINT (25496744.578 6673212.851)
2,1002,Iso-omena,"Sports Academy, 11, Piispansilta, Nuottaniemi,...","Piispansilta 11, 02230 Helsinki, Finland",POINT (25485440.532 6672255.563)
3,1003,Sello,"Mehiläinen Leppävaara, 3, Leppävaarankatu, Ete...","Leppävaarankatu 3, 02600 Helsinki, Finland",POINT (25489707.583 6678342.162)
4,1004,Jumbo,"Robert's Coffee, 3, Valuuttakatu, Pakkala, Avi...","Valuuttakatu 3, 01510 Helsinki, Finland",POINT (25498056.327 6686627.539)


In [8]:
shp.crs == geo.crs

False

In [9]:
shp.crs

<Projected CRS: EPSG:3879>
Name: ETRS89 / GK25FIN
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: Finland - 24.5°E to 25.5°E onshore nominal
- bounds: (24.5, 59.94, 25.5, 68.9)
Coordinate Operation:
- name: Finland Gauss-Kruger zone 25
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [12]:
geo = geo.to_crs(shp.crs)
geo.crs

<Projected CRS: EPSG:3879>
Name: ETRS89 / GK25FIN
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: Finland - 24.5°E to 25.5°E onshore nominal
- bounds: (24.5, 59.94, 25.5, 68.9)
Coordinate Operation:
- name: Finland Gauss-Kruger zone 25
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [28]:
geo.head()

,geometry,address
0,POINT (25497837.104 6676938.569),"13, Väinö Auerin katu, Kumpula, Helsinki, Hels..."
1,POINT (25497178.972 6675894.300),"Vallila Interior, 15, Nilsiänkatu, Vallila, He..."


In [13]:
geo.crs == shp.crs

True

In [43]:
# Find the closest shopping center for each activity location (closest shopping center from
# and closest shopping center from work) and print out the results:
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points

dest = MultiPoint(list(shp['geometry']))
orig1 = geo.at[0,'geometry']

nearest_geoms = nearest_points(orig1, dest)
type(nearest_geoms)

for idx, row in shp.iterrows():
    if row['geometry'] == nearest_geoms[1]:
        print('Closest shopping center is:', shp.at[idx,'name'])


Closest shopping center is: Center


Can you think of other application cases for the nearest neighbour analysis?